In [1]:
# @title Instalar pandas y kagglehub
!pip install --quiet pandas kagglehub


In [2]:
# @title Importar librerías y leer el CSV
import pandas as pd
from pathlib import Path
import kagglehub   # opcional si prefieres traer el archivo directo de Kaggle
import zipfile, io, requests

# ------------------------------------------------------------------
# OPCIÓN A (archivo local, p. ej. si ya subiste archive.zip a Colab)
local_zip = Path("/content/archive (1).zip")  # ajusta la ruta si cambia el nombre
if local_zip.exists():
    with zipfile.ZipFile(local_zip, "r") as zf:
        with zf.open("country_vaccinations.csv") as f:
            df = pd.read_csv(f)
else:
    # ----------------------------------------------------------------
    # OPCIÓN B (descarga con kagglehub directamente desde el dataset)
    # Requiere que tu API Token de Kaggle esté configurado
    df = kagglehub.load_dataset(
        kagglehub.KaggleDatasetAdapter.PANDAS,
        "gpreda/covid-world-vaccination-progress",
        "country_vaccinations.csv",
    )

# Vista rápida
df.head()


<ipython-input-2-98bec0fc6ebc>:18: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  df = kagglehub.load_dataset(


,country,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million,vaccines,source_name,source_website
0,Afghanistan,AFG,2021-02-22,0.0,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/
1,Afghanistan,AFG,2021-02-23,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,34.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/
2,Afghanistan,AFG,2021-02-24,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,34.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/
3,Afghanistan,AFG,2021-02-25,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,34.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/
4,Afghanistan,AFG,2021-02-26,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,34.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/


In [3]:
# @title Limpiar nombres de columnas y tipos
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")
df["date"] = pd.to_datetime(df["date"], format="%Y-%m-%d")

# Convertir “people_fully_vaccinated”, “total_vaccinations”, etc. a numérico
num_cols = [
    "total_vaccinations",
    "people_vaccinated",
    "people_fully_vaccinated",
    "daily_vaccinations",
    "total_vaccinations_per_hundred",
    "people_vaccinated_per_hundred",
    "people_fully_vaccinated_per_hundred",
]
df[num_cols] = df[num_cols].apply(pd.to_numeric, errors="coerce")

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86512 entries, 0 to 86511
Data columns (total 15 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   country                              86512 non-null  object        
 1   iso_code                             86512 non-null  object        
 2   date                                 86512 non-null  datetime64[ns]
 3   total_vaccinations                   43607 non-null  float64       
 4   people_vaccinated                    41294 non-null  float64       
 5   people_fully_vaccinated              38802 non-null  float64       
 6   daily_vaccinations_raw               35362 non-null  float64       
 7   daily_vaccinations                   86213 non-null  float64       
 8   total_vaccinations_per_hundred       43607 non-null  float64       
 9   people_vaccinated_per_hundred        41294 non-null  float64       
 10  people_ful

In [4]:
# @title Definir rango de fechas a analizar
fecha_inicio = "2021-01-01"   # cambia si lo necesitas
fecha_fin    = "2021-12-31"

mask = (df["date"] >= fecha_inicio) & (df["date"] <= fecha_fin)
df_periodo = df.loc[mask].copy()

print(f"Filas en el periodo {fecha_inicio} → {fecha_fin}: {len(df_periodo):,}")


Filas en el periodo 2021-01-01 → 2021-12-31: 68,630


In [5]:
# @title Vacunación acumulada por país (última fecha disponible en el periodo)
ultimo_dia = (
    df_periodo.groupby("country")["date"].transform("max") == df_periodo["date"]
)
pais_total = (
    df_periodo.loc[ultimo_dia]
    .sort_values("people_fully_vaccinated_per_hundred", ascending=False)
    .reset_index(drop=True)
    .loc[
        :,
        [
            "country",
            "total_vaccinations",
            "people_fully_vaccinated",
            "people_fully_vaccinated_per_hundred",
        ],
    ]
)

# Guardar a CSV
pais_total.to_csv("vacunacion_por_pais_2021.csv", index=False)

pais_total.head(15)


,country,total_vaccinations,people_fully_vaccinated,people_fully_vaccinated_per_hundred
0,Pitcairn,94.0,47.0,100.00
1,Brunei,898401.0,400691.0,90.75
2,Portugal,19328231.0,9103550.0,89.53
3,Chile,44313091.0,16543067.0,86.11
4,Cuba,30874045.0,9672464.0,85.46
5,Singapore,11583542.0,4618252.0,84.68
6,Malta,1072274.0,435874.0,84.46
7,South Korea,103880993.0,42629829.0,83.09
8,Cambodia,30456720.0,13659518.0,80.60
9,Denmark,12110043.0,4600866.0,79.14


from matplotlib import pyplot as plt
_df_0['total_vaccinations'].plot(kind='hist', bins=20, title='total_vaccinations')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['people_fully_vaccinated'].plot(kind='hist', bins=20, title='people_fully_vaccinated')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['people_fully_vaccinated_per_hundred'].plot(kind='hist', bins=20, title='people_fully_vaccinated_per_hundred')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_3.plot(kind='scatter', x='total_vaccinations', y='people_fully_vaccinated', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['index']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'index'}, axis=1)
              .sort_values('index', ascending=True))
  xs = counted['index']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_18.sort_values('index', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('index')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
_df_19['index'].plot(kind='line', figsize=(8, 4), title='index')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_20['total_vaccinations'].plot(kind='line', figsize=(8, 4), title='total_vaccinations')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_21['people_fully_vaccinated'].plot(kind='line', figsize=(8, 4), title='people_fully_vaccinated')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_22['people_fully_vaccinated_per_hundred'].plot(kind='line', figsize=(8, 4), title='people_fully_vaccinated_per_hundred')
plt.gca().spines[['top', 'right']].set_visible(False)

In [6]:
# @title Agregar por mes a nivel global
df_periodo["año_mes"] = df_periodo["date"].dt.to_period("M")
global_mensual = (
    df_periodo.groupby("año_mes")["daily_vaccinations"].sum().reset_index()
)
global_mensual["año_mes"] = global_mensual["año_mes"].dt.to_timestamp()

# Guardar a CSV
global_mensual.to_csv("vacunacion_global_mensual_2021.csv", index=False)

global_mensual


,año_mes,daily_vaccinations
0,2021-01-01,9.352253e+07
1,2021-02-01,1.709605e+08
2,2021-03-01,3.432002e+08
3,2021-04-01,5.356132e+08
4,2021-05-01,7.905041e+08
5,2021-06-01,1.137931e+09
6,2021-07-01,1.059879e+09
7,2021-08-01,1.203581e+09
8,2021-09-01,9.719360e+08
9,2021-10-01,8.141776e+08


from matplotlib import pyplot as plt
global_mensual['daily_vaccinations'].plot(kind='hist', bins=20, title='daily_vaccinations')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['año_mes']
  ys = series['daily_vaccinations']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = global_mensual.sort_values('año_mes', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('año_mes')
_ = plt.ylabel('daily_vaccinations')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['año_mes']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'año_mes'}, axis=1)
              .sort_values('año_mes', ascending=True))
  xs = counted['año_mes']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = global_mensual.sort_values('año_mes', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('año_mes')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
global_mensual['daily_vaccinations'].plot(kind='line', figsize=(8, 4), title='daily_vaccinations')
plt.gca().spines[['top', 'right']].set_visible(False)

In [7]:
# @title Top 10 países por media de vacunaciones diarias
promedio_diario = (
    df_periodo.groupby("country")["daily_vaccinations"].mean().nlargest(10)
)
promedio_diario = promedio_diario.reset_index().rename(
    columns={"daily_vaccinations": "prom_dosis_diarias"}
)

promedio_diario.to_csv("top10_promedio_diario_2021.csv", index=False)

promedio_diario


,country,prom_dosis_diarias
0,China,7.650299e+06
1,India,4.081357e+06
2,United States,1.394732e+06
3,Brazil,9.483803e+05
4,Indonesia,7.649993e+05
5,Japan,6.296164e+05
6,Vietnam,4.967665e+05
7,Pakistan,4.624017e+05
8,Germany,4.077166e+05
9,Mexico,4.073678e+05


from matplotlib import pyplot as plt
promedio_diario['prom_dosis_diarias'].plot(kind='hist', bins=20, title='prom_dosis_diarias')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
promedio_diario['prom_dosis_diarias'].plot(kind='line', figsize=(8, 4), title='prom_dosis_diarias')
plt.gca().spines[['top', 'right']].set_visible(False)